# 1 - Data Reading

In [28]:
# Data directory
workdir = "data/dataset1"

import numpy as np

# Load cleartext from the file
with open(f'{workdir}/cleartext.txt', 'r') as file:
    cleartexts = [list(map(int, line.split())) for line in file.readlines()]

# Load power traces; you have 16 trace files
traces = [np.loadtxt(f'{workdir}/trace{i}.txt') for i in range(16)]

In [29]:
print(f"cleartexts: {cleartexts[0]}")
print(f"traces: {traces[0][0]}")

cleartexts: [39, 168, 54, 116, 93, 36, 132, 228, 1, 64, 93, 235, 21, 241, 187, 168]
traces: [21337.6 19891.2 20467.2 ... 20812.8 20876.8 19379.2]


# 2 - Hypothesis Generation

In [30]:
# Example function to calculate hamming weight
def hamming_weight(n):
    return bin(n).count('1')

# Hypothetical power consumption for one key byte guess across all cleartexts
def hypothetical_power(cleartexts, key_guess):
    return [hamming_weight(ct ^ key_guess) for ct in cleartexts]

# Assume you are guessing the first key byte
key_guesses = [hypothetical_power([ct[0] for ct in cleartexts], k) for k in range(256)]


In [31]:
print(f"key_guesses: {key_guesses[0]}")

key_guesses: [4, 2, 3, 4, 6, 3, 5, 7, 3, 5, 2, 6, 2, 3, 3, 3, 5, 6, 7, 5, 3, 3, 5, 5, 6, 5, 3, 4, 5, 4, 4, 4, 2, 6, 3, 4, 3, 5, 6, 5, 4, 1, 4, 5, 2, 2, 3, 4, 5, 3, 2, 3, 4, 2, 7, 3, 1, 6, 4, 3, 4, 5, 4, 3, 4, 3, 3, 4, 5, 4, 3, 4, 1, 4, 4, 7, 4, 2, 2, 3, 2, 6, 4, 2, 5, 6, 3, 3, 5, 4, 2, 0, 5, 2, 3, 3, 4, 3, 4, 3, 5, 3, 6, 2, 2, 3, 5, 4, 6, 2, 5, 3, 3, 3, 4, 5, 2, 4, 5, 5, 2, 2, 3, 1, 2, 6, 7, 6, 5, 5, 3, 5, 3, 2, 3, 6, 3, 4, 6, 5, 3, 3, 4, 4, 4, 3, 4, 8, 2, 5]


# 3 - Correlation Analysis

In [32]:
import numpy as np

def pearson_correlation(x, y):
    if len(x) != len(y):
        print("Mismatch:", len(x), len(y))
        raise ValueError("Length of x and y must be equal.")
    return np.corrcoef(x, y)[0, 1]

def expand_hyp_power(hyp_power, num_points):
    length = len(hyp_power)
    repeats = num_points // length  # Integer division
    remainder = num_points % length  # Remainder of the division
    
    expanded_array = np.repeat(hyp_power, repeats)
    if remainder > 0:
        expanded_array = np.concatenate((expanded_array, hyp_power[:remainder]))
    
    return expanded_array

# Assuming traces is a list of arrays, each of size (150, 50000)
# Initialize the correlations array with dimensions [16, 256]
correlations = np.zeros((16, 256))  # 16 byte indexes and 256 key guesses

for byte_index in range(16):  # Assuming 16 key bytes
    print(byte_index)
    for k in range(256):  # 256 possible key guesses
        hyp_power = hypothetical_power([ct[byte_index] for ct in cleartexts], k)
        expanded_hyp_power = expand_hyp_power(hyp_power, 50000)
        
        single_correlations = []
        for trace_index in range(150):
            actual_trace = traces[byte_index][trace_index, :]
            corr = pearson_correlation(expanded_hyp_power, actual_trace)
            single_correlations.append(corr)
        
        mean_corr = np.mean(single_correlations)
        correlations[byte_index][k] = mean_corr  # Store the mean correlation for this byte and key guess

best_key_guesses = np.argmax(correlations, axis=1)
print("Best key guesses:", best_key_guesses)



0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Best key guesses: [  4 167  33 150 100 102 116  82 100 109 108 104 165 115  73  51]


In [33]:
print("Best key guesses:", best_key_guesses)
# [  4 167  33 150 100 102 116  82 100 109 108 104 165 115  73  51]

Best key guesses: [  4 167  33 150 100 102 116  82 100 109 108 104 165 115  73  51]
